In [24]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [25]:
gmaps.configure(g_key)

In [26]:
weather_data = "data/weather_data.csv"

# Read the weather data 
weather_read = pd.read_csv(weather_data)
weather_read

,Unnamed: 0,City,Country,Date,Latitude,Longitutde,Temperature (F),Humidity (%),Cloudiness (%),Windiness(mph)
0,0,Banda Aceh,ID,1586720959,5.56,95.32,79.84,78,49,5.53
1,1,Krasnosel'kup,RU,1586720960,65.70,82.47,35.24,98,100,7.02
2,2,Busselton,AU,1586720892,-33.65,115.33,61.65,66,63,8.75
3,3,Bambous Virieux,MU,1586720960,-20.34,57.76,78.89,74,75,14.99
4,4,Ushuaia,AR,1586720851,-54.80,-68.30,42.80,87,75,27.51
...,...,...,...,...,...,...,...,...,...,...
566,566,Yelovo,RU,1586721052,57.05,54.92,38.43,84,58,3.94
567,567,Ixtapa,MX,1586721052,20.70,-105.20,79.41,39,20,13.87
568,568,Upata,VE,1586721052,8.01,-62.40,89.76,31,82,9.89
569,569,Province of Trapani,IT,1586721052,37.83,12.67,57.72,93,66,4.70


In [27]:
# Store latitude and longitude in locations
location = weather_read[["Latitude","Longitutde"]]

# Fill NaN values and convert to float
humidity = weather_read["Humidity (%)"]

In [28]:
# Plot Heatmap
fig = gmaps.figure()
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
# Create heat layer
heat_layer = gmaps.heatmap_layer(location, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [95]:
new_cond = weather_read[(weather_read["Humidity (%)"] <= 40) & (weather_read["Temperature (F)"] <= 60)
                        & (weather_read["Temperature (F)"] > 40) & (weather_read["Cloudiness (%)"] <=10)]
new_cond

,Unnamed: 0,City,Country,Date,Latitude,Longitutde,Temperature (F),Humidity (%),Cloudiness (%),Windiness(mph)
176,176,Yalvaç,TR,1586720996,38.30,31.18,47.43,36,0,3.98
255,255,Mattawa,US,1586721008,46.74,-119.90,53.76,24,1,3.36
291,291,Zhangjiakou,CN,1586721013,40.81,114.88,46.17,31,0,5.03
343,343,Biłgoraj,PL,1586721020,50.54,22.72,55.40,37,0,5.82
344,344,Uyuni,BO,1586721020,-20.46,-66.83,57.06,8,0,14.20
372,372,Dolní Bousov,CZ,1586721024,50.44,15.13,55.78,33,0,6.93
459,459,Emmett,US,1586721038,43.87,-116.50,51.01,20,1,10.29
494,494,Portales,US,1586721042,34.19,-103.33,51.30,40,1,23.04
507,507,Asău,RO,1586721044,46.43,26.40,52.50,40,4,14.99
524,524,Mujiayingzi,CN,1586721046,42.12,118.78,45.97,32,0,6.13


In [96]:
fave_city = new_cond.loc[:,["City","Country","Latitude","Longitutde"]]
fave_city["Hotel Name"] = ""
fave_city

,City,Country,Latitude,Longitutde,Hotel Name
176,Yalvaç,TR,38.30,31.18,
255,Mattawa,US,46.74,-119.90,
291,Zhangjiakou,CN,40.81,114.88,
343,Biłgoraj,PL,50.54,22.72,
344,Uyuni,BO,-20.46,-66.83,
372,Dolní Bousov,CZ,50.44,15.13,
459,Emmett,US,43.87,-116.50,
494,Portales,US,34.19,-103.33,
507,Asău,RO,46.43,26.40,
524,Mujiayingzi,CN,42.12,118.78,


In [97]:
target_coordinates = "47.6062095,-122.3320708"

#redefine params
params = {
    "keyword": "lodgings",
    "radius": 5000,
    "key": g_key
}

In [98]:
#loop trought fave_city and run a lat/long search 
for index, row in fave_city.iterrows():
    
    lat = row["Latitude"]
    lng = row["Longitutde"]
    
    #change location each iteration while leaving original params in place
    params["location"]=f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_nearby = requests.get(base_url,params=params).json()
    
    try:
        fave_city.loc[index,"Hotel Name"] = hotel_nearby["results"][0]["name"]
    except: 
        print("Missing field/result...skipping")

Missing field/result...skipping
Missing field/result...skipping


In [100]:
fave_city

,City,Country,Latitude,Longitutde,Hotel Name
176,Yalvaç,TR,38.30,31.18,Hotel Psidia
255,Mattawa,US,46.74,-119.90,
291,Zhangjiakou,CN,40.81,114.88,Zhangjiakou Venice Hotel
343,Biłgoraj,PL,50.54,22.72,Karczma Na Woli
344,Uyuni,BO,-20.46,-66.83,SALCAY HOTEL
372,Dolní Bousov,CZ,50.44,15.13,Aroma Gold
459,Emmett,US,43.87,-116.50,Holiday Motel & R.V Park
494,Portales,US,34.19,-103.33,Best Western Plus Portales Inn
507,Asău,RO,46.43,26.40,Pensiunea Sophia
524,Mujiayingzi,CN,42.12,118.78,


In [101]:
print(len(fave_city))

12


In [103]:
locations = fave_city[["Latitude","Longitutde"]]
locations

,Latitude,Longitutde
176,38.30,31.18
255,46.74,-119.90
291,40.81,114.88
343,50.54,22.72
344,-20.46,-66.83
372,50.44,15.13
459,43.87,-116.50
494,34.19,-103.33
507,46.43,26.40
524,42.12,118.78


In [104]:
#add markers on gmaps where the nearest hotels are 
markers = gmaps.marker_layer(locations)

fig.add_layer(markers)

fig